In [1]:
!pip install reg_resampler
!pip install catboost

     |████████████████████████████████| 64.8MB 60kB/s 


In [74]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import mean_squared_log_error,mean_squared_error
from reg_resampler import resampler
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')

In [78]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/JanathaHack/Demand Forecasting/train.csv")
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/JanathaHack/Demand Forecasting/test.csv")
submit = pd.read_csv("/content/drive/My Drive/Colab Notebooks/JanathaHack/Demand Forecasting/sample_submission.csv")

# train = train[(train["units_sold"]<1000)]
train["week"] = pd.to_datetime(train.week,dayfirst=True)
train["year"] = train["week"].dt.year
train["month"] = train["week"].dt.month
train["week"] = train["week"].dt.weekofyear
train["price"] = train["base_price"] - train ["total_price"]
train.drop("record_ID",axis=1)
# train = train[(train["units_sold"]<1000) & (train["week"]>=29) & (train["week"]<=40)].reset_index(drop=True)


holdout = train[((train["year"]==2012)&(train["week"]>=29)&(train["week"]<=40))].drop(["year","month"],axis=1)
holdout_train = train[~((train["year"]==2012)&(train["week"]>=29)&(train["week"]<=40))].drop(["year","month"],axis=1)



test["week"] = pd.to_datetime(test.week,dayfirst=True)
test["year"] = test["week"].dt.year
test["month"] = test["week"].dt.month
test["week"] = test["week"].dt.weekofyear.astype(int)


test["price"] = test["base_price"] - test ["total_price"]
test.drop("record_ID",axis=1)

Train = train
Test = test

In [79]:
Train

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,year,month,price
0,1,3,8091,216418,99.0375,111.8625,0,0,20,2011,1,12.8250
1,2,3,8091,216419,99.0375,99.0375,0,0,28,2011,1,0.0000
2,3,3,8091,216425,133.9500,133.9500,0,0,19,2011,1,0.0000
3,4,3,8091,216233,133.9500,133.9500,0,0,44,2011,1,0.0000
4,5,3,8091,217390,141.0750,141.0750,0,0,52,2011,1,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
150145,212638,28,9984,223245,235.8375,235.8375,0,0,38,2013,7,0.0000
150146,212639,28,9984,223153,235.8375,235.8375,0,0,30,2013,7,0.0000
150147,212642,28,9984,245338,357.6750,483.7875,1,1,31,2013,7,126.1125
150148,212643,28,9984,547934,141.7875,191.6625,0,1,12,2013,7,49.8750


In [80]:
Test

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,year,month,price
0,212645,29,8091,216418,108.3000,108.3000,0,0,2013,7,0.0000
1,212646,29,8091,216419,109.0125,109.0125,0,0,2013,7,0.0000
2,212647,29,8091,216425,133.9500,133.9500,0,0,2013,7,0.0000
3,212648,29,8091,216233,133.9500,133.9500,0,0,2013,7,0.0000
4,212649,29,8091,217390,176.7000,176.7000,0,0,2013,7,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
13855,232281,40,9984,223245,241.5375,241.5375,0,0,2013,10,0.0000
13856,232282,40,9984,223153,240.8250,240.8250,0,0,2013,10,0.0000
13857,232285,40,9984,245338,382.6125,401.8500,1,1,2013,10,19.2375
13858,232286,40,9984,547934,191.6625,191.6625,0,0,2013,10,0.0000


In [81]:
x = Train.drop("units_sold",axis =1)
y = Train["units_sold"]

In [ ]:
lgb =LGBMRegressor(random_state =27)

In [ ]:
lgb.fit(x,y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=27, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [89]:
re = resampler()
Y_classes = re.fit(Train,"units_sold",bins=5)

INFO: Class 3 has been merged into Class 2 due to low number of samples

Class Distribution:
-------------------
0: 149942
1: 192
2: 16



In [90]:
kfold, scores = StratifiedKFold(n_splits=7,shuffle=False,random_state=0), list()
oof=[]
for train, test in kfold.split(x,Y_classes):
    x_train, x_test = x.iloc[train], x.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]

    model = LGBMRegressor(n_estimators=500,random_state =27)
    model.fit(x_train, np.log(y_train),categorical_feature=(1,2,3,6,7),early_stopping_rounds=20,
              eval_metric="RMSE",eval_set = [(x_test,np.log(y_test))],verbose=500)
    preds = np.exp(model.predict(x_test))
    score = 1000*np.sqrt(mean_squared_log_error(y_test, preds))
    scores.append(score)
    oof.append(np.exp(model.predict(Test)))
    print(score)
print("Average: ", sum(scores)/len(scores))

Training until validation scores don't improve for 20 rounds.
[500]	valid_0's l2: 0.131277	valid_0's rmse: 0.362322
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.131277	valid_0's rmse: 0.362322
334.69559411292613
Training until validation scores don't improve for 20 rounds.
[500]	valid_0's l2: 0.13049	valid_0's rmse: 0.361234
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.13049	valid_0's rmse: 0.361234
334.3670670214425
Training until validation scores don't improve for 20 rounds.
[500]	valid_0's l2: 0.130998	valid_0's rmse: 0.361936
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.130998	valid_0's rmse: 0.361936
334.03953132772153
Training until validation scores don't improve for 20 rounds.
[500]	valid_0's l2: 0.134903	valid_0's rmse: 0.367291
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.134903	valid_0's rmse: 0.367291
337.70995364799444
Training until validation scores don't improve for 20 r

In [ ]:
  #387 - 332.78172754309014

In [82]:
model = LGBMRegressor(n_estimators=2000,random_state =27)

In [71]:
model.fit(holdout_train.drop("units_sold",axis=1),np.log(holdout_train["units_sold"]))

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=2000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=27, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [72]:
preds = np.exp(model.predict(holdout.drop("units_sold",axis=1)))
score = 1000*np.sqrt(mean_squared_log_error(holdout["units_sold"], preds))
score

375.28564284997407

In [84]:
model.fit(x,np.log(y))

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=2000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=27, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [85]:
preds = np.exp(model.predict(Test))

In [ ]:
preds = np.mean(oof, 0)

In [86]:
submit["units_sold"] = preds

In [87]:
submit.to_csv("LGBsubmit.csv",index=False)